# 1. Load dan preprocessing data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter # Untuk melihat distribusi kelas

In [2]:
file_path = "indian_liver_patient.csv"
df = pd.read_csv(file_path)

In [3]:
print(df.info)

<bound method DataFrame.info of      Age  Gender  Total_Bilirubin  Direct_Bilirubin  Alkaline_Phosphotase  \
0     65  Female              0.7               0.1                   187   
1     62    Male             10.9               5.5                   699   
2     62    Male              7.3               4.1                   490   
3     58    Male              1.0               0.4                   182   
4     72    Male              3.9               2.0                   195   
..   ...     ...              ...               ...                   ...   
578   60    Male              0.5               0.1                   500   
579   40    Male              0.6               0.1                    98   
580   52    Male              0.8               0.2                   245   
581   31    Male              1.3               0.5                   184   
582   38    Male              1.0               0.3                   216   

     Alamine_Aminotransferase  Aspartate_Am

In [4]:
# Cek missing values dan isi dengan median jika ada
print(df.isnull().sum())

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64


In [5]:
# Pastikan 'columns' didefinisikan atau iterasi langsung pada df.columns
for col in df.columns: # Menggunakan df.columns agar lebih robust
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)

/tmp/ipython-input-5-2406694639.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


In [11]:
# Pisahkan fitur dan target
# DETEKSI "SAKIT/BERISIKO" SEBAGAI KELAS POSITIF (1) ***
# Dataset: 1 = Liver patient, 2 = Not liver patient
# Kita akan mendefinisikan: 1 = Liver patient (Sakit/Berisiko), 0 = Non-Liver patient (Sehat)
X = df.drop(columns=['Dataset']).values
y = (df['Dataset'].values == 1).astype(int).reshape(-1, 1) # Jika Dataset=1 (sakit), maka y=1. Jika Dataset=2 (sehat), maka y=0.
#Encode Gender
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
print(f"\nBentuk data X: {X.shape}")
print(f"Bentuk data y: {y.shape}")
print(f"Jumlah pasien sakit (y=1): {np.sum(y == 1)}")
print(f"Jumlah pasien sehat (y=0): {np.sum(y == 0)}")
print(f"Rasio kelas positif: {np.sum(y == 1) / len(y):.2f}")


Bentuk data X: (583, 8)
Bentuk data y: (583, 1)
Jumlah pasien sakit (y=1): 416
Jumlah pasien sehat (y=0): 167
Rasio kelas positif: 0.71


In [7]:
#Korelasi Data
cor_matrix = np.corrcoef(df.drop(columns=['Dataset']).values.T)

plt.imshow(cor_matrix, cmap='coolwarm', interpolation='none')
plt.colorbar()
plt.title("Matriks Korelasi")
plt.xticks(range(len(df.columns)-1), df.columns[:-1], rotation=90)
plt.yticks(range(len(df.columns)-1), df.columns[:-1])
plt.show()

In [8]:
df.drop(['Total_Bilirubin', 'Aspartate_Aminotransferase'], axis=1, inplace=True)

SPLIT DATA

In [12]:
print("\n--- SPLIT DATA ---")
def train_val_test_split(X, y, val_size=0.15, test_size=0.15, random_state=42):
    np.random.seed(random_state)
    n = len(X)
    idx = np.random.permutation(n)

    test_split_idx = int(n * (1 - test_size))
    val_split_idx = int(test_split_idx * (1 - val_size / (1 - test_size)))

    train_idx = idx[:val_split_idx]
    val_idx = idx[val_split_idx:test_split_idx]
    test_idx = idx[test_split_idx:]

    return X[train_idx], X[val_idx], X[test_idx], \
           y[train_idx], y[val_idx], y[test_idx]

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y)


print(f"Bentuk X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Bentuk X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"Bentuk X_test: {X_test.shape}, y_test: {y_test.shape}")



--- SPLIT DATA ---
Bentuk X_train: (407, 8), y_train: (407, 1)
Bentuk X_val: (88, 8), y_val: (88, 1)
Bentuk X_test: (88, 8), y_test: (88, 1)


In [13]:
print("\n--- Normalisasi fitur berdasarkan X_train ---")
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
std[std == 0] = 1  # Hindari pembagian dengan nol

X_train_normalized = (X_train - mean) / std
X_val_normalized = (X_val - mean) / std
X_test_normalized = (X_test - mean) / std

print(f"Bentuk X_train setelah Normalisasi: {X_train_normalized.shape}, y_train: {y_train.shape}")
print(f"Bentuk X_val setelah Normalisasi: {X_val_normalized.shape}, y_val: {y_val.shape}")
print(f"Bentuk X_test setelah Normalisasi: {X_test_normalized.shape}, y_test: {y_test.shape}")



--- Normalisasi fitur berdasarkan X_train ---
Bentuk X_train setelah Normalisasi: (407, 8), y_train: (407, 1)
Bentuk X_val setelah Normalisasi: (88, 8), y_val: (88, 1)
Bentuk X_test setelah Normalisasi: (88, 8), y_test: (88, 1)


# 2. Neural Network dengan backpropagation

In [14]:
print("\n--- 2. Membangun Neural Network ---")
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.0):
        np.random.seed(42)
        self.dropout_rate = dropout_rate

        self.weights_input_hidden = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
        self.bias_hidden = np.zeros((1, hidden_size))

        self.weights_hidden_output = np.random.randn(hidden_size, output_size) * np.sqrt(1.0 / hidden_size)
        self.bias_output = np.zeros((1, output_size))

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def sigmoid(self, x):
        x = np.clip(x, -500, 500)
        return 1 / (1 + np.exp(-x))

    def feedforward(self, X, training=True):
        self.z1 = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.a1 = self.relu(self.z1)

        if training and self.dropout_rate > 0:
            self.mask = (np.random.rand(*self.a1.shape) > self.dropout_rate) / (1 - self.dropout_rate)
            self.a1 *= self.mask
        else:
            self.mask = np.ones_like(self.a1) # Should not scale during inference

        self.z2 = np.dot(self.a1, self.weights_hidden_output) + self.bias_output
        self.a2 = self.sigmoid(self.z2)
        return self.a2

    def compute_loss(self, y_true, y_pred):
        y_true = y_true.reshape(-1, 1)  # Penting!
        y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def backward(self, X, y_true, y_pred, learning_rate):
        y_true = y_true.reshape(-1, 1)  # Penting!
        error_output = y_pred - y_true  # (batch_size, 1)
        delta_output = error_output
        error_output = y_pred - y_true
        delta_output = error_output

        d_weights_hidden_output = np.dot(self.a1.T, delta_output)
        d_bias_output = np.sum(delta_output, axis=0, keepdims=True)

        error_hidden = np.dot(delta_output, self.weights_hidden_output.T)
        delta_hidden = error_hidden * self.relu_derivative(self.z1)

        if self.dropout_rate > 0:
            delta_hidden *= self.mask # Apply mask to delta_hidden in backward pass as well

        d_weights_input_hidden = np.dot(X.T, delta_hidden)
        d_bias_hidden = np.sum(delta_hidden, axis=0, keepdims=True)

        self.weights_hidden_output -= learning_rate * d_weights_hidden_output
        self.bias_output -= learning_rate * d_bias_output
        self.weights_input_hidden -= learning_rate * d_weights_input_hidden
        self.bias_hidden -= learning_rate * d_bias_hidden

    def train(self, X_train, y_train, X_val, y_val, epochs, learning_rate, batch_size=32, patience=75):
        losses_train = []
        losses_val = []
        accuracies_train = []
        accuracies_val = []

        best_val_loss = float('inf')
        best_weights = None
        epochs_no_improve = 0

        n_samples = X_train.shape[0]

        print(f"\nMemulai pelatihan untuk {epochs} epoch...")
        for epoch in range(epochs):
            indices = np.random.permutation(n_samples)
            X_shuffled = X_train[indices]
            y_shuffled = y_train[indices]

            for start in range(0, n_samples, batch_size):
                end = start + batch_size
                xb = X_shuffled[start:end]
                yb = y_shuffled[start:end]

                y_pred_batch = self.feedforward(xb, training=True)
                self.backward(xb, yb, y_pred_batch, learning_rate)

            # Evaluate on full training set
            y_pred_train = self.feedforward(X_train, training=False)
            loss_train = self.compute_loss(y_train, y_pred_train)
            losses_train.append(loss_train)
            preds_train = (y_pred_train > 0.5).astype(int)
            acc_train = np.mean(preds_train == y_train)
            accuracies_train.append(acc_train)

            # Evaluate on validation set
            y_pred_val = self.feedforward(X_val, training=False)
            loss_val = self.compute_loss(y_val, y_pred_val)
            losses_val.append(loss_val)
            preds_val = (y_pred_val > 0.5).astype(int)
            acc_val = np.mean(preds_val == y_val)
            accuracies_val.append(acc_val)


            if epoch % 50 == 0 or epoch == epochs - 1:
                print(f"Epoch {epoch+1}/{epochs} - L_Train: {loss_train:.4f} | A_Train: {acc_train*100:.2f}% | L_Val: {loss_val:.4f} | A_Val: {acc_val*100:.2f}%")

        print("\nPelatihan selesai.")
        # Ensure best weights are loaded if training completed without early stopping
        if best_weights is not None and epochs_no_improve < patience:
             (self.weights_input_hidden,
              self.weights_hidden_output,
              self.bias_hidden,
              self.bias_output) = best_weights
        return losses_train, accuracies_train, losses_val, accuracies_val

    def predict(self, X, threshold=0.5):
        y_pred_proba = self.feedforward(X, training=False)
        return (y_pred_proba > threshold).astype(int)


--- 2. Membangun Neural Network ---


# 3. Inisialisasi dan training

In [15]:
input_size = X_train_normalized.shape[1]
hidden_size = 32
output_size = 1

nn = NeuralNetwork(input_size, hidden_size, output_size, dropout_rate=0.2)

epochs = 1000
learning_rate = 0.05
batch_size = 8

losses_train, accuracies_train, losses_val, accuracies_val = nn.train(
    X_train_normalized, y_train,
    X_val_normalized, y_val,
    epochs, learning_rate, batch_size)

y_test_pred = nn.predict(X_test_normalized)
accuracy_test = np.mean(y_test_pred == y_test)
print(f"Akurasi pada data test: {accuracy_test*100:.2f}%")


Memulai pelatihan untuk 1000 epoch...
Epoch 1/1000 - L_Train: 0.6224 | A_Train: 71.01% | L_Val: 0.7140 | A_Val: 72.73%
Epoch 51/1000 - L_Train: 0.4226 | A_Train: 77.15% | L_Val: 0.6754 | A_Val: 65.91%
Epoch 101/1000 - L_Train: 0.3852 | A_Train: 80.84% | L_Val: 0.7219 | A_Val: 68.18%
Epoch 151/1000 - L_Train: 0.3623 | A_Train: 80.84% | L_Val: 0.7365 | A_Val: 70.45%
Epoch 201/1000 - L_Train: 0.3817 | A_Train: 78.13% | L_Val: 0.9518 | A_Val: 72.73%
Epoch 251/1000 - L_Train: 0.3518 | A_Train: 80.34% | L_Val: 0.9506 | A_Val: 75.00%
Epoch 301/1000 - L_Train: 0.3576 | A_Train: 79.61% | L_Val: 1.0447 | A_Val: 72.73%
Epoch 351/1000 - L_Train: 0.3221 | A_Train: 84.77% | L_Val: 1.2512 | A_Val: 70.45%
Epoch 401/1000 - L_Train: 0.3249 | A_Train: 82.56% | L_Val: 1.2137 | A_Val: 65.91%
Epoch 451/1000 - L_Train: 0.3127 | A_Train: 84.03% | L_Val: 1.3038 | A_Val: 68.18%
Epoch 501/1000 - L_Train: 0.3124 | A_Train: 82.80% | L_Val: 1.4712 | A_Val: 73.86%
Epoch 551/1000 - L_Train: 0.3010 | A_Train: 85.01% 

In [16]:
print("\n--- 4. Evaluasi Model pada Data Test ---")

y_test_pred_proba = nn.feedforward(X_test_normalized, training=False)
y_test_pred = (y_test_pred_proba > 0.5).astype(int)

def calculate_metrics(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    accuracy = (tp + tn) / (tp + tn + fp + fn)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return accuracy, precision, recall, f1_score, {'TP': tp, 'TN': tn, 'FP': fp, 'FN': fn}

test_accuracy, test_precision, test_recall, test_f1_score, confusion_matrix_counts = \
    calculate_metrics(y_test, y_test_pred)

print(f"\nHasil Evaluasi (Kelas Positif = 'Pasien Penyakit Hati / Berisiko'):")
print(f"Akurasi pada data test: {test_accuracy * 100:.2f}%")
print(f"Precision pada data test: {test_precision * 100:.2f}%")
print(f"Recall pada data test: {test_recall * 100:.2f}%")
print(f"F1-Score pada data test: {test_f1_score * 100:.2f}%")

print("\nConfusion Matrix (jumlah):")
print("--------------------------")
print(f"True Positive (TP): {confusion_matrix_counts['TP']}  (Benar memprediksi pasien sakit)")
print(f"False Negative (FN): {confusion_matrix_counts['FN']} (Gagal memprediksi pasien sakit)")
print(f"False Positive (FP): {confusion_matrix_counts['FP']} (Salah memprediksi orang sehat jadi sakit)")
print(f"True Negative (TN): {confusion_matrix_counts['TN']} (Benar memprediksi orang sehat)")
print("--------------------------")


--- 4. Evaluasi Model pada Data Test ---

Hasil Evaluasi (Kelas Positif = 'Pasien Penyakit Hati / Berisiko'):
Akurasi pada data test: 78.41%
Precision pada data test: 78.21%
Recall pada data test: 96.83%
F1-Score pada data test: 86.52%

Confusion Matrix (jumlah):
--------------------------
True Positive (TP): 61  (Benar memprediksi pasien sakit)
False Negative (FN): 2 (Gagal memprediksi pasien sakit)
False Positive (FP): 17 (Salah memprediksi orang sehat jadi sakit)
True Negative (TN): 8 (Benar memprediksi orang sehat)
--------------------------


In [17]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(losses_train, label='Training Loss')
plt.plot(losses_val, label='Validation Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(accuracies_train, label='Training Accuracy')
plt.plot(accuracies_val, label='Validation Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [18]:
# --- 6. Contoh Deteksi untuk Pasien Baru ---
print("\n--- 6. Contoh Deteksi untuk Pasien Baru ---")

# Data pasien baru (contoh dummy data)
# Tetap gunakan data mentah, normalisasi akan dilakukan setelah ini
data_pasien_baru_1 = np.array([[
    38,    # Age - usia dewasa muda
    1,     # Gender - laki-laki
    0.6,   # Direct_Bilirubin - sedikit di atas normal (batas: <0.3 mg/dL)
    210,   # Alkaline_Phosphotase - sedang (batas: 44-147 U/L → ini agak tinggi)
    42,    # Alamine_Aminotransferase - mendekati tinggi (batas <41 U/L)
    6.5,   # Total_Protiens - dalam batas bawah normal
    3.4,   # Albumin - dalam batas bawah normal (normal: 3.5–5.0 g/dL)
    1.0    # Albumin_and_Globulin_Ratio - mendekati bawah (batas normal: 1.1–2.5)
]])



data_pasien_baru_2 = np.array([[
    30,    # Age
    0,     # Gender (0 = Female)
    0.2,   # Direct_Bilirubin
    100,   # Alkaline_Phosphotase
    20,    # Alamine_Aminotransferase
    7.5,   # Total_Protiens
    4.0,   # Albumin
    1.1    # Albumin_and_Globulin_Ratio
]])


data_pasien_baru_3 = np.array([[
    40,    # Age
    0,     # Gender (0 = Female)
    0.4,   # Direct_Bilirubin
    150,   # Alkaline_Phosphotase
    60,    # Alamine_Aminotransferase (agak tinggi)
    6.8,   # Total_Protiens
    3.2,   # Albumin
    0.9    # Albumin_and_Globulin_Ratio
]])





--- 6. Contoh Deteksi untuk Pasien Baru ---


In [19]:
# Normalisasi data pasien baru menggunakan mean dan std dari TRAIN set (yang sudah di-resample)
data_pasien_baru_1_normalized = (data_pasien_baru_1 - mean) / std
data_pasien_baru_2_normalized = (data_pasien_baru_2 - mean) / std
data_pasien_baru_3_normalized = (data_pasien_baru_3 - mean) / std


In [23]:
def prediksi_distribusi(nn, data, nama="Pasien"):
    print(f"\n--- Prediksi {nama} ---")
    prob = nn.feedforward(data, training=False)[0][0]
    print(f"Probabilitas model (pasien liver): {prob:.4f}\n")
    print("Distribusi Probabilitas:")

    kategori = [
        ("Tidak menunjukkan indikasi penyakit liver.", 0.15),
        ("Berisiko rendah mengidap penyakit liver.", 0.40),
        ("Berisiko sedang mengalami gangguan liver.", 0.60),
        ("Kemungkinan menderita penyakit liver kronis ringan-menengah.", 0.80),
        ("Indikasi penyakit liver serius, perlu pemeriksaan lanjut.", 0.95)
    ]

    # Temukan kategori utama (terdekat)
    i_utama = min(range(len(kategori)), key=lambda i: abs(prob - kategori[i][1]))

    distribusi = []
    for i, (ket, tengah) in enumerate(kategori):
        if i == i_utama:
            distribusi.append((ket, prob))
        else:
            jarak = abs(prob - tengah)
            skor = 1 / jarak if jarak != 0 else 999
            distribusi.append((ket, skor))

    # Normalisasi
    total = sum(p for i, (_, p) in enumerate(distribusi) if i != i_utama)
    sisa = 1 - prob
    distribusi = [
        (k, p if i == i_utama else (p / total) * sisa)
        for i, (k, p) in enumerate(distribusi)
    ]

    # Cetak
    for i, (keterangan, p) in enumerate(sorted(distribusi, key=lambda x: x[1], reverse=True), 1):
        print(f"{i}. {p:.4f} → {keterangan}")


In [24]:
prediksi_distribusi(nn, data_pasien_baru_1_normalized, "Pasien 1")
prediksi_distribusi(nn, data_pasien_baru_2_normalized, "Pasien 2")
prediksi_distribusi(nn, data_pasien_baru_3_normalized, "Pasien 3")



--- Prediksi Pasien 1 ---
Probabilitas model (pasien liver): 0.5384

Distribusi Probabilitas:
1. 0.5384 → Berisiko sedang mengalami gangguan liver.
2. 0.2078 → Berisiko rendah mengidap penyakit liver.
3. 0.1099 → Kemungkinan menderita penyakit liver kronis ringan-menengah.
4. 0.0740 → Tidak menunjukkan indikasi penyakit liver.
5. 0.0699 → Indikasi penyakit liver serius, perlu pemeriksaan lanjut.

--- Prediksi Pasien 2 ---
Probabilitas model (pasien liver): 0.4606

Distribusi Probabilitas:
1. 0.4606 → Berisiko rendah mengidap penyakit liver.
2. 0.2515 → Berisiko sedang mengalami gangguan liver.
3. 0.1129 → Tidak menunjukkan indikasi penyakit liver.
4. 0.1033 → Kemungkinan menderita penyakit liver kronis ringan-menengah.
5. 0.0716 → Indikasi penyakit liver serius, perlu pemeriksaan lanjut.

--- Prediksi Pasien 3 ---
Probabilitas model (pasien liver): 0.5551

Distribusi Probabilitas:
1. 0.5551 → Berisiko sedang mengalami gangguan liver.
2. 0.1847 → Berisiko rendah mengidap penyakit liver